In [1]:
import sqlite3
import numpy as np
import mysql.connector
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
plotly.tools.set_credentials_file(username='mainakchain', api_key='B9m4DVrYYmsoW3jDSiok')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
from scipy import stats
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import datetime
import holidays
from sklearn.model_selection import cross_val_score
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
import operator

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [168]:
time = pd.read_csv('time_between_two_tolls_new.csv')

In [242]:
toll_freq = dict(time['toll_both'].value_counts())
time['toll_freq'] = time['toll_both']
time['toll_freq'] = time['toll_freq'].apply(lambda x: toll_freq[x] )
m = int(0.3 * time['toll_freq'].median())
time = time[(time['toll_freq'] > m)]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [247]:
time.shape

(3582, 14)

In [227]:
time.toll_both.value_counts()

Gurapalli                                   183
Madapam                                     181
Manguli                                     180
Gudipada (Old Gangapada)                    179
Chilakapalem                                179
Sergrarh                                    179
Nathavalasa/ Vizianagaram                   178
Laxmipuram                                  178
Panikholi                                   178
Krishnavaram                                175
Budanam                                     175
Vempadu                                     174
Kalaparru                                   174
Kaza                                        174
Pottipadu                                   171
Sunambatti (Musunur)                        171
Bolapalli                                   171
Tangutur                                    167
Bellupada                                   156
Gaddurur                                    131
Mulbagal                                

In [215]:
toll_time = pd.read_csv('toll_time.csv')

In [216]:
toll_time = toll_time[toll_time['origin'] == 'Jamshedpur'][toll_time['destination'] == 'HOSUR']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [224]:
toll_time.toll_both.value_counts()

Manguli                                     47
Vempadu                                     47
Gurapalli                                   46
Panikholi                                   46
Krishnavaram                                45
Gudipada (Old Gangapada)                    45
Kalaparru                                   45
Madapam                                     45
Pottipadu                                   44
Sunambatti (Musunur)                        44
Laxmipuram                                  44
Chilakapalem                                44
Nathavalasa/ Vizianagaram                   44
Bolapalli                                   44
Budanam                                     43
Venkatachallam &#160;(Old Nellore)          43
Sergrarh                                    43
Kaza                                        43
Tangutur                                    40
Mulbagal                                    35
Gaddurur                                    35
Bellupada    

In [31]:
toll_time.toll_both.nunique()

33

In [33]:
# toll_time = time
# toll_time.drop(['Unnamed: 0'], axis=1,inplace=True)

In [34]:
toll_time

,origin,destination,toll_both,vehicle_no,loading_out_time,odometer_km,ist_timestamp,distance_from_toll,loading_in_time,start_date,eta,unloading_in_time,slug,delivery_status,distance_travelled,late_by_hrs,time_taken
3904,Jamshedpur,HOSUR,Aganampudi,JH01AE6027,2017-11-06 00:16:58,7915.73,2017-11-10 22:14:15,0.898483,2017-11-04 20:05:00,2017-11-04 00:00:00,2017-11-13 00:00:00,2017-11-13 01:39:39,ETJ0E000,early,624.41,-18.422500,117.954722
3905,Jamshedpur,HOSUR,Aganampudi,TN36Y9906,2017-11-04 23:23:59,6810.38,2017-11-07 01:37:32,0.805438,2017-11-04 05:45:21,2017-11-04 00:00:00,2017-11-16 00:00:00,2017-11-10 03:37:24,ETOYD000,early,627.52,-146.132500,50.225833
3906,Jamshedpur,HOSUR,Attibele (BETL),BR02GA3943,2017-12-01 00:18:48,9133.44,2017-12-04 21:51:11,0.977303,2017-11-30 05:35:54,2017-11-30 00:00:00,2017-12-08 00:00:00,2017-12-05 18:44:15,ETF9G000,early,1610.77,-58.860833,93.539722
3907,Jamshedpur,HOSUR,Attibele (BETL),OD07M1151,2017-11-30 04:58:30,9912.20,2017-12-03 23:40:14,0.672070,2017-11-29 05:00:12,2017-11-29 00:00:00,2017-12-07 00:00:00,2017-12-04 00:00:14,ET46G000,early,1641.28,-76.999444,90.695556
3908,Jamshedpur,HOSUR,Attibele (BETL),OR04M5329,2017-11-27 03:32:37,7302.32,2017-11-30 23:16:39,0.902463,2017-11-25 05:57:26,2017-11-25 00:00:00,2017-12-07 00:00:00,2017-12-04 00:00:53,ETXSF000,early,1620.49,-77.942500,91.733889
3909,Jamshedpur,HOSUR,Attibele (BETL),OR07V0091,2017-11-27 03:16:12,11154.36,2017-11-30 22:57:04,0.861116,2017-11-25 06:03:58,2017-11-25 00:00:00,2017-12-07 00:00:00,2017-12-04 00:14:03,ETZSF000,early,1611.30,-77.831944,91.681111
3910,Jamshedpur,HOSUR,Attibele (BETL),OR07W2988,2017-11-26 22:59:12,5659.67,2017-12-02 00:41:59,0.957378,2017-11-25 14:02:01,2017-11-25 00:00:00,2017-12-04 00:00:00,2017-12-04 08:19:42,ETHUF000,on-time,1615.10,-5.705278,121.713056
3911,Jamshedpur,HOSUR,Attibele (BETL),OR11J1747,2017-11-24 23:00:25,5947.85,2017-12-02 22:12:30,0.896532,2017-11-24 06:20:40,2017-11-24 00:00:00,2017-12-02 00:00:00,2017-12-03 00:00:42,ETDPF000,late,1625.30,17.667222,191.201389
3912,Jamshedpur,HOSUR,Attibele (BETL),WB23B7298,2017-11-30 14:58:09,6820.49,2017-12-05 01:34:25,0.931364,2017-11-28 11:02:52,2017-11-28 00:00:00,2017-12-06 00:00:00,2017-12-05 12:52:17,ETU3G000,early,1509.52,-22.176389,106.604444
3913,Jamshedpur,HOSUR,Bellupada,AP04TX2171,2017-11-22 19:30:49,8130.89,2017-11-24 03:29:09,0.250613,2017-11-22 14:50:16,2017-11-22 00:00:00,2017-12-04 00:00:00,2017-11-28 05:08:28,ETAHF000,early,376.19,-153.696667,31.972222


In [18]:
# toll_time.drop(['Unnamed: 0'], axis=1, inplace=True)

In [35]:
new_toll_time = toll_time.copy()

In [36]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

In [37]:
new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

In [38]:
new_toll_time = toll_time.copy()

In [39]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

new_toll_time['unloading_in_time'] = new_toll_time['unloading_in_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['avg_travel_time'] = new_toll_time['start_date'] + (new_toll_time['unloading_in_time'] - new_toll_time['start_date'])/2

new_toll_time['month_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.month)

new_toll_time['ist_timestamp'] = new_toll_time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
new_toll_time['time_of_travel'] = new_toll_time.ist_timestamp.apply(lambda x: x.hour)

new_toll_time['day_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.weekday())

new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x: x.hour)

In [40]:
new_toll_time = new_toll_time[['toll_both','vehicle_no','loading_out_time','distance_travelled','time_of_travel','day_of_travel','month_of_travel','days_from_now','time_taken']]

In [151]:
# toll_dict = {'Sergrarh':1,'Panikholi':2, 'Manguli':3, 'Gudipada (Old Gangapada)':4, 'Gurapalli':5, 'Bellupada':6, 'Madapam':7, 'Chilakapalem':8, 'Nathavalasa/ Vizianagaram':9, 'Laxmipuram':10, 'Vempadu':11, 'Krishnavaram':12, 'Kalaparru':13, 'Pottipadu':14, 'Kaza':15, 'Bolapalli':16, 'Sunambatti (Musunur)':17, 'Venkatachallam &#160;(Old Nellore)':18, 'Budanam':19, 'Gaddurur':20, 'Mulbagal':21, 'Hoskote':22, 'Attibele (BETL)':23, 'Aganampudi':24}

In [46]:
def find_toll_num(df):
    toll_dict = {}
    new_toll_dict = {}
    num = 1
    for toll in df.toll_both.unique().tolist():
        d = df['distance_travelled'][df.toll_both == toll].mean()
        toll_dict[toll] = d
    
    for key, value in sorted(toll_dict.iteritems(), key=lambda (k,v): (v,k)):
        new_toll_dict[key] = num
        num+=1
        
    return new_toll_dict

In [48]:
new_toll_dict = find_toll_num(new_toll_time)

In [65]:
train_df = new_toll_time.copy()
test_df= pd.DataFrame(columns=['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now', 'time_taken'])
choice = True
while(choice):
    today_date = datetime.datetime.now()
    vehicle = raw_input('Enter the vehicle no.: ')
    start = raw_input('Enter the start date in %Y-%m-%d %H:%M:%S format: ')
    start_delta = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    
    loading_out = raw_input('Enter the loading out time in %Y-%m-%d %H:%M:%S format: ')
    loading_out_delta = datetime.datetime.strptime(loading_out, '%Y-%m-%d %H:%M:%S')
    
    a = []
    loop = input('Enter the number of tolls vehicle crossed sequentitially: ')
    count = 0
    print '      '
    while (count != loop):
        b = []
        toll = raw_input('Toll both: ')
        distance = input('Distance travelled from 1st toll in km: ')
        ist = raw_input('IST Timestamp in %Y-%m-%d %H:%M:%S format: ')
        ist_delta = datetime.datetime.strptime(ist, '%Y-%m-%d %H:%M:%S')
        b.extend([toll, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])
        a.append(b)
        count+=1
    
    add_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now', 'time_btn_tolls'])
    train_df = train_df.append(add_df)
    
    test_toll = raw_input('Enter the numbers of tolls the vehicle shall take in the journey sequentially: ')
    test_toll_list = test_toll.split(',')
    test_toll_list = [str(a) for a in test_toll_list]

    a = []

    for toll_val in test_toll_list:
        b = []
        distance = toll_time['distance_travelled'][toll_time.toll_both == toll_val].median()
        ist_delta = loading_out_delta + timedelta(hours=toll_time['time_taken'][toll_time.toll_both == toll_val].median())
        b.extend([toll_val, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])

        a.append(b)
    add_test_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now','time_btn_tolls'])
    
    test_df = test_df.append(add_test_df)
    
    c = raw_input('Want to enter more vehicles?(y/n)')
    
    if (c == 'y' or c == 'Y'):
        continue
    elif (c == 'n' or c == 'N'):
        choice = False
        print 'bbye'
    else:
        print 'Wrong Choice! Exiting'
        choice = False

Enter the vehicle no.: OR04M532M
Enter the start date in %Y-%m-%d %H:%M:%S format: 2017-11-02 00:00:00
Enter the loading out time in %Y-%m-%d %H:%M:%S format: 2017-11-03 12:02:43
Enter the number of tolls vehicle crossed sequentitially: 3
      
Toll both: Sergrarh
Distance travelled from 1st toll in km: 0
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 08:22:51
Toll both: Panikholi
Distance travelled from 1st toll in km: 95.1
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 10:42:12
Toll both: Manguli
Distance travelled from 1st toll in km: 146.8
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 12:10:12
Enter the numbers of tolls the vehicle shall take in the journey sequentially: Gudipada (Old Gangapada),Gurapalli,Bellupada,Madapam,Chilakapalem,Nathavalasa/ Vizianagaram,Laxmipuram,Vempadu,Krishnavaram,Kalaparru,Pottipadu,Kaza,Bolapalli,Sunambatti (Musunur),Venkatachallam &#160;(Old Nellore),Budanam,Gaddurur,Mulbagal,Hoskote,Attibele (BETL)
Want to enter more vehicles?

In [66]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [72]:
train_df.to_csv('final_train_df_14Dec2017.csv')
test_df.to_csv('final_test_df_14Dec2017.csv')

In [73]:
train_df.shape, test_df.shape

((998, 9), (38, 9))

In [74]:
# test_df

In [75]:
# def change_toll_booth_to_dummy(df):
#     num = 1
#     dict_toll_both = {}
#     for toll in df['toll_both'].unique().tolist():
#         df['toll_both'] = df['toll_both'].replace({toll: num})
#         dict_toll_both[num] = toll
#         num += 1
#     return dict_toll_both

In [76]:
# dict_toll_both_train_df = change_toll_booth_to_dummy(train_df)
train_df.toll_both = train_df.toll_both.replace(new_toll_dict)
test_df.toll_both = test_df.toll_both.replace(new_toll_dict)

In [77]:
def change_vehicle_no_to_dummy(df):
    num = 1
    dict_vehicle_no = {}
    for truck in df['vehicle_no'].unique().tolist():
        df['vehicle_no'] = df['vehicle_no'].replace({truck: num})
        dict_vehicle_no[num] = truck
        num += 1
    return dict_vehicle_no

In [78]:
dict_vehicle_no_train_df = change_vehicle_no_to_dummy(train_df)

In [79]:
rev_dict_vehicle_no_test_df = dict((v,k) for k,v in dict_vehicle_no_train_df.iteritems())

In [80]:
test_df.vehicle_no = test_df.vehicle_no.replace(rev_dict_vehicle_no_test_df)

In [81]:
train_df['virtual_avg_speed'] = train_df['distance_travelled'] / train_df['time_taken']
test_df['virtual_avg_speed'] = test_df['distance_travelled'] / test_df['time_taken']

In [82]:
test_df_copy = test_df.copy()

In [83]:
test_df.drop(['time_taken'], axis=1, inplace=True)

In [163]:
# MODEL BUILDING AND APPLICATION

In [84]:
train_X = train_df.drop(['time_taken'], axis=1)
train_y = train_df.time_taken

In [85]:
train_X.shape

(998, 9)

In [86]:
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.3, random_state=42)

In [87]:
reg_mlp = MLPRegressor(hidden_layer_sizes=(100,50), activation='logistic', alpha=8, solver='lbfgs', random_state=42)

In [88]:
scale= MinMaxScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

In [89]:
reg_mlp.fit(X_train_scaled, y_train)
reg_mlp.score(X_test_scaled, y_test) * 100

88.360236166104116

In [90]:
scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.97 (+/- 0.03)


In [91]:
scale = MinMaxScaler()
X_predict_scaled = scale.fit_transform(test_df)

In [92]:
predictions = reg_mlp.predict(X_predict_scaled)

In [127]:
show_df = test_df.copy()
show_df['predicted'] = predictions

In [128]:
show_df['predict_diff'] = show_df['predicted'].diff()

In [129]:
def med_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].median()

In [130]:
def mean_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].mean()

In [131]:
show_df['med_time'] = show_df['toll_both']
show_df['med_time'] = show_df.med_time.apply(lambda x:med_toll_time_taken(x))
show_df['mean_time'] = show_df['toll_both']
show_df['mean_time'] = show_df.mean_time.apply(lambda x:mean_toll_time_taken(x))

In [132]:
show_df['avg_pred_med'] = (show_df['predicted'] + show_df['med_time']) / 2

In [133]:
x = train_df
x['avg_speed'] = x.distance_travelled/ x.time_taken
x = x[x.vehicle_no == 25].sort_values('distance_travelled')
x['avg_speed'] = x.distance_travelled/(x.time_taken - 40.88)
x

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,virtual_avg_speed,avg_speed
793,2,25,23,0.00,16,3,11,25,40.885833,0.000000,0.000000
704,3,25,23,90.70,21,3,11,25,45.974444,1.972835,17.803708
576,4,25,23,147.17,5,3,11,25,53.906389,2.730103,11.297836
235,5,25,23,204.06,6,3,11,25,55.247222,3.693579,14.203163
280,6,25,23,292.31,9,3,11,25,58.063611,5.034306,17.010976
27,7,25,23,377.35,13,3,11,25,61.943889,6.091804,17.914546
528,8,25,23,493.02,17,3,11,25,66.076667,7.461333,19.566874
154,9,25,23,520.71,18,3,11,25,66.876667,7.786124,20.029876
657,11,25,23,560.93,19,3,11,25,67.979444,8.251465,20.698948
484,10,25,23,560.93,19,3,11,25,67.979444,8.251465,20.698948


In [134]:
train_df[train_df['vehicle_no'] == 45]

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,virtual_avg_speed,avg_speed
990,2,45,12,0.0,8,5,11,40,20.335556,0.000000,0.000000
991,3,45,12,95.1,10,5,11,40,22.658056,4.197183,4.197183
992,4,45,12,146.8,12,5,11,40,24.124722,6.085044,6.085044


In [135]:
show_df['avg_speed_pred'] = show_df['distance_travelled']/show_df['predicted']
show_df['avg_speed_med'] = show_df['distance_travelled']/show_df['med_time']

In [136]:
show_df['test_pred'] = show_df['predicted'] + show_df.avg_speed_pred

In [137]:
show_df['round_pred'] = show_df['predicted'].round()

In [138]:
show_df['median_input'] =  test_df_copy['time_taken']

In [139]:
show_df.drop(['virtual_avg_speed','predict_diff','mean_time','avg_pred_med','avg_speed_med','avg_speed_pred'], axis=1, inplace=True)

In [157]:
show_df['mean_pred_med'] = (show_df['round_pred'] + show_df['median_input'] )/2

In [158]:
train_df[(train_df['toll_both'] == 33) | (train_df['toll_both'] == 32)]

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,virtual_avg_speed,avg_speed
2,33,3,0,1610.77,21,5,12,14,93.539722,17.220171,17.220171
3,33,4,4,1641.28,23,4,12,15,90.695556,18.096587,18.096587
4,33,5,3,1620.49,23,2,11,19,91.733889,17.665118,17.665118
5,33,6,3,1611.30,22,2,11,19,91.681111,17.575049,17.575049
6,33,7,22,1615.10,0,2,11,19,121.713056,13.269735,13.269735
7,33,8,23,1625.30,22,1,11,20,191.201389,8.500461,8.500461
8,33,9,14,1509.52,1,4,12,16,106.604444,14.160010,14.160010
302,32,3,0,1574.09,18,5,12,14,90.166111,17.457668,17.457668
303,32,13,4,1578.05,5,5,11,29,144.914167,10.889550,10.889550
304,32,1,0,1584.41,0,2,11,40,168.228056,9.418227,9.418227


In [160]:
new_toll_dict

{'Aganampudi': 13,
 'Attibele (BETL)': 33,
 'Bellupada': 7,
 'Bolapalli': 21,
 'Budanam': 26,
 'Chilakapalem': 9,
 'Eethakota': 16,
 'Gaddurur': 27,
 'Gudipada (Old Gangapada)': 5,
 'Gurapalli': 6,
 'Hasanpur (Sainkul)': 1,
 'Hoskote': 32,
 'Kalaparru': 18,
 'Kaza': 20,
 'Krishnagiri': 31,
 'Krishnavaram': 15,
 'Laxmipuram': 10,
 'Madapam': 8,
 'Main Toll (Gosthani Gate) (Sheela Nagar)': 12,
 'Manguli': 4,
 'Mulbagal': 28,
 'Nathavalasa/ Vizianagaram': 11,
 'Palempalli': 23,
 'Pallikonda': 29,
 'Panikholi': 3,
 'Pottipadu': 19,
 'Sergrarh': 2,
 'Sunambatti (Musunur)': 24,
 'Tangutur': 22,
 'Unguturu': 17,
 'Vaniyambadi': 30,
 'Vempadu': 14,
 'Venkatachallam &#160;(Old Nellore)': 25}

In [159]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,med_time,test_pred,round_pred,median_input,mean_pred_med
0,5,45,12,203.320,10,6,11,40,40.966541,45.996250,45.929616,41.0,46.387500,43.693750
1,6,45,12,291.250,13,6,11,40,37.942090,49.686667,45.618262,38.0,49.882639,43.941319
2,7,45,12,375.245,20,6,11,40,51.406044,56.755556,58.705672,51.0,56.755556,53.877778
3,8,45,12,490.530,0,0,11,40,55.744775,60.303056,64.544344,56.0,60.303056,58.151528
4,9,45,12,517.415,3,0,11,40,61.819924,63.638889,70.189637,62.0,63.638889,62.819444
5,11,45,12,556.620,5,0,11,40,64.102620,65.382917,72.785885,64.0,65.382917,64.691458
6,10,45,12,556.620,5,0,11,40,63.573588,65.382917,72.329111,64.0,65.382917,64.691458
7,14,45,12,684.270,11,0,11,40,68.525055,71.403333,78.510745,69.0,71.403333,70.201667
8,15,45,12,753.930,16,0,11,40,74.526918,76.884444,84.643129,75.0,76.884444,75.942222
9,18,45,12,897.510,2,1,11,40,84.239283,86.089444,94.893576,84.0,86.089444,85.044722
